In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LOAD THE LIBRARIES

In [ ]:
%%time
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp
import json
import glob
import kagglehub
import matplotlib.pyplot as plt

# LOAD THE GEMMA MODEL

In [ ]:
%%time
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en") # didn't work well
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_7b_en") # can't allocate memory for this model

In [ ]:
gemma_lm.summary()

# LOAD THE DATA

In [ ]:
# wildcard to match all JSON files in the folder. Note- this is the synthetic data from chatgpt
json_files = glob.glob('/kaggle/input/qa-json-pairs-final/*.json')
print(json_files)

dataframes = []

for file in json_files[1:4]:
    df = pd.read_json(file)
    dataframes.append(df)

qa_pairs = pd.concat(dataframes, ignore_index=True)

# load the qa dataset generated using "FoodieFinder_GenerateQADataset" Kaggle notebook
qa_pairs_cat_data = pd.read_csv("/kaggle/input/qa-json-pairs-final/qa_df.csv")
qa_pairs_cat_data.rename(columns={"Question": "question", "Answer": "answer"}, inplace=True)

qa_pairs = pd.concat([qa_pairs,qa_pairs_cat_data])
qa_pairs.shape

In [ ]:
qa_pairs.head()

In [ ]:
# recommended_sushi_places.json has list values in it, therefore loading the data in for this file differently
with open("/kaggle/input/qa-json-pairs-final/recommended_sushi_places.json", "r") as file:
    data = json.load(file)

# Flatten the 'answer' field if it's a list and create a list of dictionaries
flattened_data = []
for entry in data['qa_pairs']:
    question = entry['question']
    # If the answer is a list, join it into a single string
    if isinstance(entry['answer'], list):
        answer = " ".join(entry['answer'])
    else:
        answer = entry['answer']
    flattened_data.append({'question': question, 'answer': answer})

# Convert to DataFrame
sushi_df = pd.DataFrame(flattened_data)

qa_pairs = pd.concat([qa_pairs,sushi_df])

In [ ]:
qa_pairs.shape

# ADD VARIATIONS TO THE QA DATASET

In [ ]:
# Create question templates with variations to train the model on rephrased versions of the same question.
question_patterns = {
    "What are some recommended dishes to try": [
        "Can you suggest some dishes to try",
        "What dishes would you recommend",
        "Which dishes are a must-try",
    ],
    "Recommend a place to try": [
        "Could you suggest a place to try",
        "What are some good places to try",
        "Where should I go to try this",
    ],
    "What are some restaurants": [
        "Could you recommend some restaurants",
        "What are a few good restaurants",
        "Any recommendations for restaurants",
    ],
    "What are some good places to try": [
        "Where can I find some good places to try",
        "Can you recommend some good places",
        "What are a few top places to try",
    ],
    "What restaurants are known for": [
        "Which restaurants are famous for",
        "Do you know of any restaurants known for",
        "Are there restaurants that specialize in",
    ],
    "What is the average rating of": [
        "How is the average rating for",
        "What's the rating like for",
        "Can you tell me the average rating of",
    ],
    "What is the location of": [
        "Where is it located?",
        "Can you share the location of",
        "Where can I find",
    ],
    "What is the full address of": [
        "Could you provide the full address for",
        "What's the complete address of",
        "Can I have the full address of",
    ],
    "What is the price normally spent for dining at the restaurant": [
        "What's the average cost for dining at",
        "How much is usually spent for a meal at",
        "Can you tell me the typical price range for dining at",
    ],
    "Recommend a restaurant that specializes in": [
        "Could you suggest a place that specializes in",
        "Do you know any restaurants that offer",
        "Where can I go for a restaurant that serves",
    ],
    "Tell me something about the restaurant": [
        "Can you share some information about",
        "What should I know about",
        "Could you give me some details about",
    ]
}

# Generate variations
qa_pairs_with_variations = []

for idx, row in qa_pairs.iterrows():
    question = row['question']
    answer = row['answer']

    for pattern, variations in question_patterns.items():
        if question.startswith(pattern):
            # Generate variations based on the pattern
            for variation in variations:
                # Replace the pattern with the variation in the question text
                rephrased_question = question.replace(pattern, variation, 1)
                qa_pairs_with_variations.append({"question": rephrased_question, "answer": answer})
            break
    else:
        # If no pattern matches, keep the original
        qa_pairs_with_variations.append({"question": question, "answer": answer})

# Convert to DataFrame and save as new CSV
variations_df = pd.DataFrame(qa_pairs_with_variations)
variations_df.shape

In [ ]:
variations_df.to_csv("variations_df.csv", index=False)

In [ ]:
qa_pairs = pd.concat([qa_pairs,variations_df])
qa_pairs.shape

In [ ]:
qa_pairs.dropna(inplace=True)
qa_pairs.shape

# CHECK AVERAGE TOKEN LENGTH

Below is a function to tokenize and compute average length of token. Will be useful to set the sequence length parameter while fine tuning gemma.

In [ ]:
# %%time
# from keras_nlp.models import GemmaTokenizer

# # Load the tokenizer specific to your Gemma model (assuming 'gemma_2b_en')
# tokenizer = GemmaTokenizer.from_preset('gemma_2b_en')

# # function to tokenize and compute average length
# def get_average_token_length(qa_dataset):
#     token_lengths = []
#     for index, row in qa_dataset.iterrows():
#         # Combine question and answer to tokenize them together
#         text = row['question'] + " " + row['answer']
        
#         # Tokenize using Gemma's tokenizer
#         tokens = tokenizer.tokenize(text)
        
#         # Append the token length of the current row
#         token_lengths.append(len(tokens))
    
#     # Return the average token length
#     return sum(token_lengths) / len(token_lengths) if len(token_lengths) > 0 else 0


# # Calculate the average token length
# average_length = get_average_token_length(qa_pairs)
# print(f"Average token length: {average_length}")


In [ ]:
qa_pairs.to_csv("final_qa_pairs_dataset.csv", index = False)

# DEEFINE THE TEMPLATE

In [ ]:
template = "Question:\n{question}\n\nAnswer:\n{answer}"

In [ ]:
data = []
for index, row in qa_pairs.iterrows():
    formatted_string = template.format(question=row['question'], answer=row['answer'])
    data.append(formatted_string)

# LoRA FINE TUNING

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 5
gemma_lm.backbone.enable_lora(rank=5)
gemma_lm.summary()

In [ ]:
%%time

gemma_lm.preprocessor.sequence_length = 120 #100,512,256

optimizer = keras.optimizers.AdamW(
    learning_rate = 4e-5, # 2e-4, 3e-4
    weight_decay = 0.01, #0.02
)


optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])


gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

It takes 10 hours 30 mins for fine tuning the model.

In [ ]:
%%time
history = gemma_lm.fit(data, epochs=90, batch_size=5)  # 100,60

In [ ]:
# Plot accuracy
plt.plot(history.history['sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Plot loss
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()


In [ ]:
#!pip install -q -U kagglehub --upgrade # run if required

# SAVE THE MODEL TO OUTPUT DIRECTORY

In [ ]:
%%time

## Save the finetuned model as a KerasNLP preset.
preset_dir = "./foodie_finder_gemma"
gemma_lm.save_to_preset(preset_dir)

**Note:** After saving the model, I uploaded my model manually instead of running the 2 blocks of code below.

# UPLOAD THE MODEL TO KAGGLE HUB

In [ ]:
# import kagglehub
# kagglehub.login()

In [ ]:
# %%time
# # upload to kagglehub
# kaggle_uri = f"kaggle://kjeevan/foodie_finder_gemma/keras/foodie_finder_gemma"
# keras_nlp.upload_preset(kaggle_uri, preset_dir)

# CODE TO UPLOAD THE MODEL TO HUGGING FACE

In [ ]:
# from huggingface_hub import HfApi
# from huggingface_hub import login
# from huggingface_hub import upload_folder

# # Paste the access token or use secrets from "Add-ons" for best practice
# login("the_access_token_from_hf")

In [ ]:
# repo_name = "Jeevan18/FoodieFinderV2"  # Change to your desired repository name
# HfApi().create_repo(repo_name, exist_ok=True)

In [ ]:
# %%time

# # Directory where the fine tuned model is saved
# model_dir = "/kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/foodie_finder_gemma"

# # Upload all files in the model directory to the Hugging Face Hub
# upload_folder(
#     repo_id="Jeevan18/FoodieFinderV2",  # the repository name
#     folder_path=model_dir,
#     #path_in_repo=" ",  # Upload to the root of the repo
#     commit_message="Initial model upload",
# )

# TESTING THE FINE TUNED MODEL

In [ ]:
%%time

prompt = template.format(
    question="What are some tips for visiting ramen shops in Japan?",
    answer="" 
)

# Generate the answer using the fine-tuned Gemma model
print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are the different food types served in Tokyo?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are some recommended sushi places to try in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What makes Tokyo Sushi Ginza Sushi-Ichi special?",
    answer=""
)

# Generate the answer using the fine-tuned Gemma model
print(gemma_lm.generate(prompt, max_length=800))


In [ ]:
%%time

prompt = template.format(
    question="What are some must-visit ramen spots in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))


In [ ]:
%%time

prompt = template.format(
    question="Recommend a place to enjoy a wide selection of sake in Tokyo.",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are some recommended dishes to try in Chugoku?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=1800))


In [ ]:
%%time

prompt = template.format(
    question="Recommend a place to try shabushabu in Japan.",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are some restaurants that sell good tempura in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What sets Okinawa Darumasoba apart?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are the main differences in ramen styles across Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="I want to eat sushi, where should I dine at in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="I want to eat sushi, what is a recommended sushi place in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What restaurants are known for friendly and attentive staff?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What restaurants are known for great service?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What restaurants are known for warm and inviting atmosphere and cleanliness?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Tell me something about the restaurant Tempura Yokota.",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Recommend a restaurant that specializes in steak.",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Recommend a restaurant that specializes in tempura.",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Recommend a restaurant that specializes in horumon (bbq offel).",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are some tips for visiting ramen shops in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are some must-visit ramen spots in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What are the main differences in ramen styles across Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Which restaurants are famous for friendly and attentive staff and beautiful ambiance?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="What's the average cost for dining at Ajuta?",
    answer="" 
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="How is the average rating for Ajuta?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Where can I find Ajuta in Japan?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Could you provide the full address for Ajuta?",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

In [ ]:
%%time

prompt = template.format(
    question="Can you share some information about Ajuta.",
    answer=""
)

print(gemma_lm.generate(prompt, max_length=800))

# CODE TO ADD MEMORY

Note: The memory functionality has been commented out as it caused inconsistent and unreliable chatbot responses. Further experimentation might be required before making it publicly available.

In [ ]:
# # Code to load the model from inputs section
# gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/foodie_finder_gemma")  # gemma_instruct_2b_en

In [ ]:
# def display_chat(prompt, response):
#   '''Displays an LLM prompt and response in a pretty way.'''
#   prompt = prompt.replace('\n\n','<br><br>')
#   prompt = prompt.replace('\n','<br>')
#   formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
#   response = response.replace('•', '  *')
#   response = textwrap.indent(response, '', predicate=lambda _: True)
#   response = response.replace('\n\n','<br><br>')
#   response = response.replace('\n','<br>')
#   response = response.replace("```","")
#   formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
#   return Markdown(formatted_prompt+formatted_text)

In [ ]:
# class ChatState():
#   """
#   Manages the conversation history for a turn-based chatbot
#   Follows the turn-based conversation guidelines for the Gemma family of models
#   documented at https://ai.google.dev/gemma/docs/formatting
#   """
#   def __init__(self, model):
#     """
#     Initializes the chat state.
#     Args:
#         model: The language model to use for generating responses.
#         system: (Optional) System instructions or bot description.
#     """
#     self.model = model
#     self.tokenizer = keras_nlp.models.GemmaTokenizer.from_preset("gemma2_instruct_2b_en") # /kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/foodie_finder_gemma
#     self.history = []
#   def add_to_history_as_user(self, message):
#     """
#     Adds a user message to the history with start/end turn markers.
#     """
#     self.history.append(message)
#   def add_to_history_as_model(self, message):
#     """
#     Adds a model response to the history with start/end turn markers.
#     """
#     # remove new lines
#     message = message.replace("\n"," ")
#     self.history.append(message )
#   def get_history(self):
#     """
#     Returns the entire chat history as a single string.
#     """
#     return "".join([*self.history])
#   def get_history_blurb(self):
#     """
#     Returns what to insert into the current prompt
#     """
#     if len(self.history)==0:
#       return ""
#     else:
#       return \
# f"""\n\nUse the following chat history context to respond to the instruction below:\n"""\
# f"""{self.get_history()}"""

#   def get_full_prompt(self):
#     """
#     Builds the prompt for the language model, including history and system description.
#     """
#     prompt = self.get_history()
#     return prompt
#   def send_message(self, message):
#     """
#     Handles sending a user message and getting a model response.
#     Args:
#         message: The user's message.
#     Returns:
#         The model's response.
#     """
#     # Step 2: Fake retrieving context from Chroma
#     #chroma_context = "Some people are allergic to aspirin. "
#     # Step 3: Fake retrieving web search context
#     #web_context = "Many drugs have harmful interactions if taken together. "
#     # Step 4: Construct prompt with both Chroma and web search contexts
#     prompt = self.get_full_prompt()
#     full_prompt = \
# f"""You are an AI assistant that responds to instructions about food."""\
# f"""{self.get_history_blurb()}\n"""\
# f"""Instruction: {message}"""\
# f"""Response:"""
#     # GW for debugging - print("--->\n" + full_prompt + "<--")
#     self.add_to_history_as_user(message)
#     # Generate response with full prompt
#     response = self.model.generate(full_prompt, max_length=1024)
#     # GW for debugging - print("--->\n" + response + "<--")
#     result = response.replace(full_prompt, "")  # Extract only the new response
#     # Add the result to chat history
#     self.add_to_history_as_model(result)

#     return result

In [ ]:
# chat = ChatState(gemma_lm)

In [ ]:
# %%time
# prompt = "Can I find tempura in Japan?"
# response = chat.send_message(prompt)
# display_chat(prompt, response)

In [ ]:
# %%time
# prompt = "What food are we discussing?"
# response = chat.send_message(prompt)
# display_chat(prompt, response)